# Python Client Use

This notebook demonstrates simple use of this plugin via the Steamship client.

In [ ]:
!pip install --upgrade steamship==2.2.0b2

In [ ]:
# NOTE!
# You will likely want to set your own STEAMSHIP_PROFILE here, or change it to None!
STEAMSHIP_PROFILE="staging"

# This should match the value found in the `handle` field of `steamship.json`
PLUGIN_HANDLE="trainable-tagger-default"

In [ ]:
# Create our client.
from steamship import Steamship, PluginInstance, File
ship = Steamship(profile=STEAMSHIP_PROFILE, workspace="hi-there")
from os import path

print(f"You are operating within workspace: {ship.config.workspace_handle or 'default'}")

In [ ]:
# Steamship plugins are designed to be configured at instantiation.
# The resulting plugin (and it's configuration) then remain for repeated use.
pi = ship.use_plugin(PLUGIN_HANDLE, config={
    "tag_kind": "TestLabel",
    "single_or_multi_label": "single"
})

print(f"Your plugin handle is: {pi.handle}")

In [ ]:
# Upload a file with training data
with open(path.join('..', 'test_data', 'spam-data.csv'), 'r') as f:
    file = File.create(ship, content=f.read())

In [ ]:
# Create a blockifier to parse the CSV file
blockifier = ship.use_plugin('csv-blockifier', config={
    "text_column": "Message",
    "tag_columns": "Category",
    "tag_kind": "Category"
})
print(f"Your csv blockifier handle is: {blockifier.handle}")

In [ ]:
blockify_task = file.blockify(blockifier.handle)
print(f"Your blockify task ID is {blockify_task.task_id}")

In [ ]:
# All plugin operations are asynchronous.
# Calling .wait() will wait until a timeout is reached or until we have a result
blockify_task.wait()

In [ ]:
# Now we create an exporter, which will provide us data for training
exporter = ship.use_plugin('signed-url-exporter')

In [ ]:
from steamship.plugin.inputs.training_parameter_plugin_input import TrainingParameterPluginInput
from steamship.plugin.inputs.export_plugin_input import ExportPluginInput

training_request = TrainingParameterPluginInput(
    plugin_instance=pi.handle,
    export_plugin_input=ExportPluginInput(
        plugin_instance=exporter.handle, type="file", query=f'blocktag and kind "Category"',
    ),
    training_params={},
)
train_response = pi.train(training_request)

In [ ]:
print(pi.client.config.space_handle)
print(pi.handle)

It will take about 5 hours for training to complete fully. Then you can run:

In [ ]:
tag_task = pi.tag("I like how easy this is!")
print(f"Your task ID is {tag_task.task.task_id}")

In [ ]:
tag_task.wait()